In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.utils as np_utils
from keras import backend as K
import matplotlib.pyplot as plt
import sys
import urllib
import os
import tensorflow_probability as tfp

Using TensorFlow backend.


In [2]:
####### download mnist file from website 
if not os.path.isfile("train-images-idx3-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz","train-images-idx3-ubyte.gz")
if not os.path.isfile("train-labels-idx1-ubyte.gz"):
    urllib.request.urlretrieve("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz","train-labels-idx1-ubyte.gz")

######## reading from the .gz files
import gzip
f=gzip.open("train-images-idx3-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2051:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
H=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
buf=f.read(4)
W=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_img=np.frombuffer(f.read(H*W*N),dtype=np.uint8)
train_img=train_img.reshape(N,H,W,1)
f.close()

f=gzip.open("train-labels-idx1-ubyte.gz",'rb')
buf=f.read(4)
magic_num=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))
if magic_num != 2049:
    print("invalid file ... exiting...")
    exit()
buf=f.read(4)
N=np.frombuffer(buf,dtype=np.dtype(np.int32).newbyteorder('>'))[0]
train_label=np.frombuffer(f.read(N),dtype=np.uint8)
train_label=train_label.reshape(N,1)
f.close()
train_label=np_utils.to_categorical(train_label);

In [3]:
if K.image_data_format() == 'channels_last':
    train_img = train_img.reshape(train_img.shape[0],train_img.shape[1], train_img.shape[2], train_img.shape[3])
else:
    train_img = train_img.reshape(train_img.shape[0], 1, train_img.shape[1], train_img.shape[2])
print(train_img.shape)

(60000, 28, 28, 1)


In [29]:
def new_model(m,n,o,b):
    model=Sequential()
    model.add(Conv2D(filters=5,kernel_size=(5,5),padding='Valid',activation='relu',input_shape=(28,28,1)));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(filters=5,kernel_size=(5,5),padding='Valid',activation='relu'));
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    if b==1:
        model.add(Dense(n,activation='sigmoid',use_bias=True))
    else:
        model.add(Dense(n,activation='sigmoid',use_bias=False))
    model.add(Dense(o[1],activation='softmax',use_bias=False))
    return model
def get_input(model,train_img,n):
    get_layer=K.function([model.layers[0].input],[model.layers[n].output])
    t_img=get_layer([train_img])[0]
    return t_img
def random_weight(a):
    c=a.copy
    c=-1+np.random.random(a.shape)*2
    return c


In [37]:
use_bias=1

model=new_model(train_img.shape,300,train_label.shape,use_bias)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_img,train_label,epochs=0,batch_size=100)
model.summary()
l=model.get_weights()


for i in range(len(l)):
    l[i]=random_weight(l[i])
    
model.set_weights(l)
t_img=get_input(model,train_img,len(model.layers)-2)
t_img=tf.Session().run(tf.matmul(tfp.math.pinv(t_img),train_label))
l[len(l)-1]=t_img
model.set_weights(l)
model.evaluate(train_img,train_label)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 24, 24, 5)         130       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 12, 12, 5)         0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 8, 8, 5)           630       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 4, 4, 5)           0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 80)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 300)               24300     
_________________________________________________________________
dense_24 (Dense)             (None, 10)                3000      
Total para

[2.0729782861709594, 0.5602166666666667]